In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## Padding a 3D mesh

This notebook shows how to pad a 3D mesh.

In [ ]:
import numpy as np
from nanomesh import Volume

n = 24
m = 12

data = np.ones([2*n, 5*n, 3*n], dtype=int)
data[m:3*m, m:3*m, 0:3*n] = 0

print(data.shape)

vol = Volume(data)
vol.show_slice()

# generate contours
from nanomesh.mesh3d import Mesher3D

mesher = Mesher3D(vol)
mesher.generate_contour()

# tetramesh
tetras = mesher.tetrahedralize(opts='-pAq1.2 -a100')
tetras.metadata['labels'] = tetras.metadata['tetgenRef']
tetras.plot_pyvista(jupyter_backend='static', show_edges=True)  # Use .plot_itk() for an interactive view 

### Padding a mesh

In [ ]:
import copy
from nanomesh.mesh_utils_3d import pad3d

padded_mesh = copy.copy(tetras)
padded_mesh = pad3d(padded_mesh, side='top', width=20, opts='-pAq1.2 -a100', label=3)

padded_mesh

# padded_mesh.plot_pyvista(jupyter_backend='static', show_edges=True)